In [1]:
#只是通过rnn_classification来学习rnn/LSTM，它作分类并不如cnn,因为这里我们分类只利用了某一个时间点的信息，
#和这个时间点跟其他时间点的联系来做分类预测，而没有用到时间顺序的关系
import torch
import torchvision

import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(1)    # reproducible

In [2]:
# Hyper Parameters
EPOCH = 1           # 训练整批数据多少次, 为了节约时间, 我们只训练一次
BATCH_SIZE = 64
TIME_STEP = 28      # rnn 时间步数 / 图片高度
INPUT_SIZE = 28     # rnn 每步输入值 / 图片每行像素
LR = 0.01           # learning rate

In [3]:
# Mnist 手写数字
train_data = torchvision.datasets.MNIST(
    root='./MNIST/data1/MNIST/',    # 保存或者提取位置
    train=True,  # this is training data
    transform=torchvision.transforms.ToTensor(),    # 转换 PIL.Image or numpy.ndarray 成
                                                    # torch.FloatTensor (C x H x W), 训练的时候 normalize 成 [0.0, 1.0] 区间
    download=False,          # 没下载就下载, 下载了就不用再下了
)

In [4]:
# 批训练 64samples, 1 channel, 28x28 (64, 1, 28, 28)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

In [9]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        
        #经过rnn后，原来输入的dim为64*28*28（batch,time_step,input_size），输出后的r_out的dim变为64*28*64
        self.rnn = nn.LSTM(     # 这里使用RNN的变种LSTM，因为训练效果好的多，
                                # 但是本质还是一样的，连参数设置都没变，只是最后state有两个(h_n,h_c)
            input_size=28,      # 图片每行的数据像素点
            hidden_size=64,     # rnn hidden unit
            num_layers=1,       # 有几层 RNN layers
            batch_first=True,   # RNN的默认输入为(time_step,batch, input_size) ,CNN为(batch, time_step, input_size)，转换成CNN的统一格式
        )
        
        self.out = nn.Linear(64, 10)    # 输出层

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size),其中outpu_size和hidden_size的size是一样的
                #在这里，r_out的dim为64*28*64，表示LSTM对batch中每一个28*28的图片，都分成了28个步骤处理，每个步骤处理完后输出64个元素
        # h_n shape (n_layers, batch, hidden_size)   LSTM 有两个 hidden states, h_n 是主线, h_c 是分线
        # h_c shape (n_layers, batch, hidden_size)   # n_layers可以近似看作有几个RNN（每个RNN各自的layer）并行运作
        r_out, (h_n, h_c) = self.rnn(x,None)   # None 表示 hidden state 会用全0的 state
                                               #下一个r_out都是由上一个x图片和(h_n,h_c)一起推理得到的，输出的(h_n,h_c)是最终训练得到的值

        # 选取最后一个时间点的 r_out 输出,也可以选一个其他的时间点输出（我选中间点）
        # 一个时间点代表一行，用一行，借助行与行的联系，推理得到out
        out = self.out(r_out[:, 14, :]) #r_out[:, 14, :]的dim为64*64， 这里 r_out[:, -1, :] 的值也是 h_n 的值，最终out是64*10的
        return out

In [10]:
def pred_rate(preds,labels):
    return preds.eq(labels).sum().item()/labels.shape[0]

In [11]:
device=torch.device("cuda" if torch.cuda.is_available else "cpu")
rnn = RNN().to(device)
print(rnn)

RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)


In [12]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all parameters
loss_func = nn.CrossEntropyLoss()   # the target label is not one-hotted

# training and testing
for epoch in range(EPOCH):
    for step, (x, b_y) in enumerate(train_loader):   # gives batch data
        b_x = x.view(-1, 28, 28).to(device)   # reshape x to (batch, time_step, input_size),x是64*1*28*28的，b_y的dim是64的
                                              # 现在产生需要的b_x，dim是64*28*28的。注意：这里的view函数跟reshape功能一模一样
        b_y=b_y.to(device)
        output = rnn(b_x)               # rnn output,output 的dim为64*10
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients
        
        if step%50 ==0:
            preds=output.argmax(dim=1)
            print('Epoch: ', epoch, '| Step: ', step,'| loss: ',loss.item(), '| prection rate: ',pred_rate(preds,b_y))
            
#发现准确率没有CNN高，但是它有其他方面的优点

Epoch:  0 | Step:  0 | loss:  2.3308446407318115 | prection rate:  0.09375
Epoch:  0 | Step:  50 | loss:  0.789395272731781 | prection rate:  0.75
Epoch:  0 | Step:  100 | loss:  0.28616219758987427 | prection rate:  0.90625
Epoch:  0 | Step:  150 | loss:  0.35792720317840576 | prection rate:  0.890625
Epoch:  0 | Step:  200 | loss:  0.40037187933921814 | prection rate:  0.890625
Epoch:  0 | Step:  250 | loss:  0.3135119080543518 | prection rate:  0.90625
Epoch:  0 | Step:  300 | loss:  0.22473421692848206 | prection rate:  0.9375
Epoch:  0 | Step:  350 | loss:  0.3979053795337677 | prection rate:  0.84375
Epoch:  0 | Step:  400 | loss:  0.3482958972454071 | prection rate:  0.875
Epoch:  0 | Step:  450 | loss:  0.16231508553028107 | prection rate:  0.953125
Epoch:  0 | Step:  500 | loss:  0.3202798366546631 | prection rate:  0.875
Epoch:  0 | Step:  550 | loss:  0.19951753318309784 | prection rate:  0.921875
Epoch:  0 | Step:  600 | loss:  0.44497206807136536 | prection rate:  0.875
Ep

In [28]:
# test
test_data = torchvision.datasets.MNIST(root='./MNIST/data1/MNIST/', train=False)
test_X = test_data.test_data.type(torch.FloatTensor)/255   # shape (10000, 28, 28) value in range(0,1),因为之前像素点都是0~255的
test_y = test_data.test_labels.data.numpy()   # covert to numpy array

test_X = test_X.to(device)

test_pred = rnn(test_X)   # dim 10000 * 10
test_pred = torch.max(test_pred,1)[1]

test_pred = test_pred.cpu().data.numpy()            # 先回到cpu再编程numpy数组形式

print("The prediction rate of 10000 exemples is: ",np.sum(test_pred==test_y)/10000)

The prediction rate of 10000 exemples is:  0.935
